# COCO format annotation & Detectron2 instance segmentation

## CSV to COCO format

### pycocotools

In [3]:
!pip install -Uqqq pycocotools

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pycocotools import mask as maskUtils
from joblib import Parallel, delayed
import json,itertools

### CSV 데이터 확인

In [5]:
data_dir = '/kaggle/input/sartorius-cell-instance-segmentation'
train_csv_path = '/kaggle/input/sartorius-cell-instance-segmentation/train.csv'

In [ ]:
df = pd.read_csv(train_csv_path)
df.head()

### 제반 함수
- 이번 데이터는 train set이 적어 validation 또한 train으로 사용

In [7]:
import numpy as np
from pycocotools import mask as maskUtils
from joblib import Parallel, delayed
from tqdm import tqdm

def rle2mask(rle, img_w, img_h):
    # RLE(Run Length Encoding) 문자열을 (2, N) 형태의 배열로 변환
    array = np.fromiter(rle.split(), dtype=np.uint)  # RLE 문자열을 숫자 배열로 변환
    array = array.reshape((-1, 2)).T  # (N, 2) 배열을 (2, N)으로 reshape
    array[0] = array[0] - 1  # RLE 시작점을 0부터 시작하도록 변환
    
    # RLE 인코딩을 풀어서 실제 마스크 생성 (예: [3, 1, 10, 2] -> [3, 4, 10, 11, 12])
    # 마스크를 더 빠르게 생성하기 위해 사용
    starts, lengths = array  # 시작점과 길이 분리
    mask_decompressed = np.concatenate([np.arange(s, s + l, dtype=np.uint) for s, l in zip(starts, lengths)])
    
    # 바이너리 마스크 생성
    msk_img = np.zeros(img_w * img_h, dtype=np.uint8)  # 이미지 크기만큼 0으로 채운 배열 생성
    msk_img[mask_decompressed] = 1  # RLE 인코딩에서 나온 값들에 1 할당 (마스크 영역)
    msk_img = msk_img.reshape((img_h, img_w))  # 1차원 배열을 이미지 크기(H, W)로 reshape
    msk_img = np.asfortranarray(msk_img)  # pycocotools에서 이 배열을 다룰 수 있도록 포트란 방식으로 배열 정렬
    
    return msk_img

def annotate(idx, row, cat_ids):
    # RLE 형식의 마스크를 바이너리 마스크로 변환
    mask = rle2mask(row['annotation'], row['width'], row['height'])  # RLE에서 바이너리 마스크 생성
    c_rle = maskUtils.encode(mask)  # 마스크를 다시 COCO 형식의 RLE로 인코딩
    c_rle['counts'] = c_rle['counts'].decode('utf-8')  # RLE를 바이너리에서 UTF-8로 변환
    area = maskUtils.area(c_rle).item()  # 마스크의 영역(픽셀 수) 계산
    bbox = maskUtils.toBbox(c_rle).astype(int).tolist()  # 마스크에서 바운딩 박스 계산
    
    # 각 이미지를 위한 COCO 어노테이션 생성
    annotation = {
        'segmentation': c_rle,  # COCO 형식의 RLE 세그멘테이션
        'bbox': bbox,  # 바운딩 박스 좌표 [x, y, width, height]
        'area': area,  # 마스크 영역의 면적
        'image_id': row['id'],  # 이미지 ID
        'category_id': cat_ids[row['cell_type']],  # 카테고리 ID (셀 타입)
        'iscrowd': 0,  # crowd가 아닌 개체로 설정
        'id': idx  # 어노테이션 ID
    }
    return annotation

def coco_structure(df, image_dir, workers=4):
    # COCO 형식의 카테고리와 이미지 정보를 생성하는 함수
    ## 헤더 부분(카테고리 정보) 생성
    cat_ids = {name: id + 1 for id, name in enumerate(df.cell_type.unique())}  # 각 셀 타입에 대해 ID 부여
    cats = [{'name': name, 'id': id} for name, id in cat_ids.items()]  # COCO 카테고리 형식의 리스트 생성

    # 이미지 정보 생성 (이미지 ID, 크기, 파일 경로 포함)
    images = [{'id': id, 'width': row.width, 'height': row.height, 
               'file_name': f'{image_dir}/train/{id}.png'} for id, row in df.groupby('id').agg('first').iterrows()]

    ## 어노테이션 생성
    annotations = Parallel(n_jobs=workers)(delayed(annotate)(idx, row, cat_ids) for idx, row in tqdm(df.iterrows(), total=len(df)))
    
    # COCO 형식의 딕셔너리 반환 (카테고리, 이미지, 어노테이션)
    return {'categories': cats, 'images': images, 'annotations': annotations}


### rle 변환 여부 확인

In [ ]:
rle = df.loc[0, 'annotation']
print(rle)
plt.imshow(rle2mask(rle, 704, 520));

In [ ]:
annotations = coco_structure(df, data_dir)

In [ ]:
annotations['annotations'][0]

### Annotation json 저장

In [11]:
annotation_train_json_path = "/kaggle/working/annotation_train.json"

with open(annotation_train_json_path, 'w', encoding='utf-8') as f:
    json.dump(annotations, f, ensure_ascii=True, indent=4)

### Sanity check

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

In [ ]:
coco = COCO(annotation_train_json_path)
imgIds = coco.getImgIds()

In [ ]:
imgs = coco.loadImgs(imgIds[-3:])
_, axs = plt.subplots(len(imgs), 2, figsize=(40, 15 * len(imgs)))
for img, ax in zip(imgs, axs):
    I = Image.open(os.path.join(data_dir, img['file_name']))
    annIds = coco.getAnnIds(imgIds=[img['id']])
    anns = coco.loadAnns(annIds)
    ax[0].imshow(I)
    ax[1].imshow(I)
    plt.sca(ax[1])
    coco.showAnns(anns, draw_bbox=True)

## Detectron2 Mask R-CNN

### Detectron2 설치

In [ ]:
%%time
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' -q

In [ ]:
import os
import cv2
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, DatasetMapper
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
setup_logger()

### 데이터셋 및 annotation 경로 지정

In [ ]:
train_image_dir = "/kaggle/input/sartorius-cell-instance-segmentation/train"
annotation_json = "/kaggle/working/annotation_train.json"

register_coco_instances("cell_train", {}, annotation_json, train_image_dir)

# 메타데이터에 클래스 명 설정
metadata = MetadataCatalog.get("cell_train").thing_classes = ['shsy5y', 'astro', 'cort']
dataset_dicts = DatasetCatalog.get("cell_train")

### 학습을 위한 설정 구성

In [23]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cell_train",)
cfg.DATASETS.TEST = ()  # test set이 없을 경우 빈 리스트
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # 사전 학습된 모델 가중치
cfg.SOLVER.IMS_PER_BATCH = 2  # batch size
cfg.SOLVER.BASE_LR = 0.00025  # 학습률
cfg.SOLVER.MAX_ITER = 100    # 학습 iteration 수
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # RoI batch size
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 클래스 수
cfg.INPUT.MASK_FORMAT = "bitmask"    # RLE 마스크

### 학습

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)  # output 경로 및 덮어쓰기 여부
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### 추론

In [ ]:
import os
import cv2
import random
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Test 데이터셋 경로 설정
test_image_dir = "/kaggle/input/sartorius-cell-instance-segmentation/test"
test_images = os.listdir(test_image_dir)

# 1. 학습된 모델 설정
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 클래스 수와 동일하게 설정
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 학습된 가중치 파일 경로
cfg.MODEL.DEVICE = "cuda"  # GPU 사용, CPU 사용 시 "cpu"로 설정

# 2. Predictor 생성
predictor = DefaultPredictor(cfg)

# 3. 테스트 이미지에서 예측 수행 및 시각화
for img_file in random.sample(test_images, 3):  # 테스트 이미지에서 5개 샘플링
    img_path = os.path.join(test_image_dir, img_file)
    img = cv2.imread(img_path)  # 이미지 불러오기
    
    # 예측 수행
    outputs = predictor(img)
    
    # 예측 결과 시각화
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get("cell_train"), scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # matplotlib을 사용하여 결과 이미지 시각화
    plt.figure(figsize=(12, 8))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()